# df['Detectors'] Cleaning 

I start with 133 unique Detector types, and I want to consolidate or drop. 
1. Drop anything past the semicolon (;) - redundant or miss-filled field  - DONE, put in DF creation step (this was done on import)--> 118
2. Convert "PSI" Manufacturer into "Dectris - the model is identifying --> 117
3. Convert ADSC detectors with/without "R" ending to unified with R --> 114
4. DON'T DROP rows with Detector = "NULL" --> 114
   These may be cryo-EM pdbs in the future? Filter later
5. Drop Detector types with count < 100 --> 20 detectors

In [2]:
import pandas as pd

In [3]:
df = pd.read_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/1_initial_dataframe_163K_full.feather')

In [4]:
# I on't always want to do this - Drop rows with NULL detector
# Only for some

df = df[df['Detector'] != "NULL"]

In [5]:
df['Detector'].count()

155711

In [6]:
# Replace mis-used PSI phrase in Pilatus
df['Detector'] = df['Detector'].replace('PSI PILATUS 6M', 'DECTRIS PILATUS 6M')

df['Detector'] = df['Detector'].replace('MARMOSAIC 300 MM CCD', 'RAYONIX MX-300')


In [7]:
#Correct lack of R on ADSC names
df['Detector']= df['Detector'].replace({
    'ADSC QUANTUM 315': 'ADSC QUANTUM 315R',
    'ADSC QUANTUM 4': 'ADSC QUANTUM 4R',
    'ADSC QUANTUM 210': 'ADSC QUANTUM 210R'})

In [8]:
#Detectors are better, fewer drops
det_counts = df['Detector'].value_counts()
df = df[~df['Detector'].isin(det_counts[det_counts < 100].index)]

#lots of crap in Optics that will get dropped here
optic_counts = df['Optics'].value_counts()
df = df[~df['Optics'].isin(det_counts[det_counts < 100].index)]


In [9]:
#df['Detector'].nunique()
df.head(10)

,index,Resolution,Completeness,I_sigma,R_value,R_free,Detector,Det_type,Optics
5,9ICH,2.90,92.0,NULL,0.176,NULL,SDMS,AREA DETECTOR,NULL
6,966C,1.90,NULL,NULL,NULL,0.330,ADSC,IMAGE PLATE,COLLIMATOR
7,9ICI,3.10,94.0,NULL,0.169,NULL,SDMS,AREA DETECTOR,NULL
9,9ICK,2.70,89.0,NULL,0.164,NULL,SDMS,AREA DETECTOR,NULL
11,9ICJ,3.10,88.0,NULL,0.149,NULL,SDMS,AREA DETECTOR,NULL
12,9NSE,2.24,NULL,11.6000,NULL,0.254,ADSC QUANTUM 1,CCD,MIRROR
13,9ICY,3.00,94.0,NULL,0.181,NULL,SDMS,AREA DETECTOR,NULL
14,9ICN,3.00,86.0,NULL,0.150,NULL,SDMS,AREA DETECTOR,NULL
16,9ICO,2.90,86.0,NULL,0.152,NULL,SDMS,AREA DETECTOR,NULL
17,9ICX,2.60,89.0,NULL,0.178,NULL,SDMS,AREA DETECTOR,NULL


In [10]:
df = df.reset_index()
df.to_feather('/Users/jesse/TDI_bootcamp/capstone_PDB/dataframe_feather/2_detclean_dataframe.feather')

In [11]:
df['Optics'].count()

149448